In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
import pandas as pd
import json
# Any results you write to the current directory are saved as output.

In [2]:
print ("Read Dataset ... ")
def read_dataset(path):
	return json.load(open(path)) 
train = read_dataset('train.json')
test = read_dataset('test.json')

# Text Data Features
print ("Prepare text data of Train and Test ... ")
def generate_text(data):
	text_data = [" ".join(doc['ingredients']).lower() for doc in data]
	return text_data 

train_text = generate_text(train)
test_text = generate_text(test)
target = [doc['cuisine'] for doc in train]

for i in range(len(train_text)-1):
    train_text[i] = train_text[i].replace('salt','')

# Feature Engineering 
print ("TF-IDF on text data ... ")
tfidf = TfidfVectorizer(binary=True, norm='l2')
# tfidf = CountVectorizer(binary=True)
def tfidf_features(txt, flag):
    if flag == "train":
    	x = tfidf.fit_transform(txt)
    else:
	    x = tfidf.transform(txt)
    x = x.astype('float32')
    return x 

X = tfidf_features(train_text, flag="train")
X_test = tfidf_features(test_text, flag="test")

Read Dataset ... 
Prepare text data of Train and Test ... 
TF-IDF on text data ... 


In [3]:
# Label Encoding - Target 
print ("Label Encode the Target Variable ... ")
lb = LabelEncoder()
y = lb.fit_transform(target)
# X_scaled = preprocessing.scale(X.toarray())
# test_scaled = preprocessing.scale(X_test.toarray())


Label Encode the Target Variable ... 


In [9]:
%%time
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X.toarray())
print(pca.n_components_)

1100
Wall time: 27.2 s


In [5]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y , random_state = 0,test_size=0.2)

In [8]:
%%time
from sklearn.svm import NuSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
m2 = SVC(C=100, # penalty parameter, setting it to a larger value 
                 kernel='rbf', # kernel type, rbf working fine here
                 degree=3, # default value, not tuned yet
                 gamma=2, # kernel coefficient, not tuned yet
                 coef0=1, # change to 1 from default value of 0.0
                 shrinking=True, # using shrinking heuristics
                 tol=0.0001, # stopping criterion tolerance 
                 probability=False, # no need to enable probability estimates
                 class_weight=None, # all classes are treated equally 
                 verbose=0, # print the logs 
                 max_iter=-1, # no limit, let it run
                 decision_function_shape=None, # will use one vs rest explicitly 
                 random_state=None)

# model = BaggingClassifier(m2, max_samples=0.5, max_features=0.5)
# model = AdaBoostClassifier(base_estimator=m2,algorithm='SAMME',learning_rate=0.1)
# model = OneVsRestClassifier(classifier, n_jobs=-1)

m2.fit(x_train,y_train)
print(m2.score(x_test,y_test))

0.805908233815
CPU times: user 4min 45s, sys: 993 ms, total: 4min 46s
Wall time: 4min 46s


In [6]:
%%time
from lightgbm import LGBMClassifier
from sklearn.ensemble import BaggingClassifier
m1 = LGBMClassifier(n_estimators=3000, max_depth=15)

# model = BaggingClassifier(m1, max_samples=0.8, max_features=0.8)
m1.fit(x_train, y_train)
print(m1.score(x_test,y_test))

ModuleNotFoundError: No module named 'lightgbm'

In [7]:
%%time
print ("Train the model ... ")
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
model = XGBClassifier(n_estimators=1000)

# model = OneVsRestClassifier(classifier, n_jobs=-1)
# model = OneVsOneClassifier(classifier, n_jobs=4)
model.fit(x_train, y_train)
print(model.score(x_test,y_test))


Train the model ... 


KeyboardInterrupt: 

In [26]:
len(Y_val[0])

20

In [40]:
## DNN Solution
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPool1D, GlobalAveragePooling1D, BatchNormalization, ActivityRegularization
from keras.optimizers import RMSprop, SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.cross_validation import train_test_split
Y = to_categorical(y, num_classes=20)
random_seed = 2
# Split the train and the validation set for the fitting
X_train, X_val, Y_train, Y_val = train_test_split(X.toarray(), Y, test_size = 0.2, random_state=random_seed)

model = Sequential()
model.add(Dense(256, activation='tanh', input_shape=(3051,)))
# model.add(ActivityRegularization(l1=0.0, l2=0.2))
model.add(Dense(256, activation = "tanh"))
model.add(Dense(128, activation = "tanh"))
# model.add(ActivityRegularization(l1=0.0, l2=0.2))
model.add(Dropout(0.2))
model.add(Dense(64, activation = "tanh"))
model.add(Dropout(0.6))
model.add(Dense(20, activation = "softmax"))

optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=1e-04)
optimizer1 = Adam(lr=0.001, decay=1e-4)
optimizer2 = SGD(lr=0.1, decay=1e-4)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=0.01, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.0001)
earlystopping = EarlyStopping(monitor='val_acc', patience=0.001, verbose=1, mode='auto')

In [42]:
history = model.fit(X_train, Y_train,
                      epochs=200,
                      batch_size=512,
                      validation_data=(X_val,Y_val),
                      verbose=1,callbacks=[learning_rate_reduction])

Train on 31819 samples, validate on 7955 samples
Epoch 1/200
31819/31819 [==============================] - 1s 26us/step - loss: 0.9334 - acc: 0.7551 - val_loss: 1.1942 - val_acc: 0.7032
Epoch 2/200
31819/31819 [==============================] - 1s 25us/step - loss: 0.9322 - acc: 0.7550 - val_loss: 1.1942 - val_acc: 0.7023
Epoch 3/200
31819/31819 [==============================] - 1s 27us/step - loss: 0.9266 - acc: 0.7547 - val_loss: 1.1889 - val_acc: 0.7077
Epoch 4/200
31819/31819 [==============================] - 1s 25us/step - loss: 0.9222 - acc: 0.7560 - val_loss: 1.1888 - val_acc: 0.7060
Epoch 5/200
31819/31819 [==============================] - 1s 27us/step - loss: 0.9202 - acc: 0.7569 - val_loss: 1.1890 - val_acc: 0.7048
Epoch 6/200
31819/31819 [==============================] - 1s 24us/step - loss: 0.9218 - acc: 0.7558 - val_loss: 1.1814 - val_acc: 0.7045
Epoch 7/200
31819/31819 [==============================] - 1s 27us/step - loss: 0.9208 - acc: 0.7545 - val_loss: 1.1820 - v

KeyboardInterrupt: 